# CART and the Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import os

## Import data

In [ ]:
# where is the working directory set?
os.getcwd()

In [ ]:
col_admit = pd.read_csv("../assets/dataset/collegeadmissions.csv")

col_admit.head(10)

## Parse data

In [ ]:
# check parsing
col_admit.info()

In [ ]:
# re-parse
col_admit['admit'] == 1

# create dummy variables
rank_dummy = pd.get_dummies(col_admit['rank'], prefix='rank')

# fuse back
col_admit_clean = pd.concat([col_admit, rank_dummy], axis = 1)

# drop original rank column
col_admit_clean = col_admit_clean.drop(columns = ['rank'])

In [ ]:
# check parsing
col_admit_clean.info()

In [ ]:
# looks ok
col_admit_clean.head(10)

## Model

### Training

Tree & Random Forest

In [ ]:
# specifying classifiers - Tree & Random Forest
clf = tree.DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators = 100, oob_score = True, random_state = 123456)

# feature set
X = col_admit_clean.drop(columns = ['admit', "rank_1"])

# target
y = col_admit_clean['admit']

# creating training / testings datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print(len(X_train.index))
print(len(X_test.index))

In [ ]:
# fit the classifier# fit t 
clf.fit(X_train,y_train)

In [ ]:
rf.fit(X_train, y_train)

### Testing

In [ ]:
# predicted
predicted = clf.predict(X_test)

# predicted probabilities
predictied_prob = clf.predict_proba(X_test)

# actual
validate = pd.DataFrame(y_test)

validate.columns = ['actual']

validate['predicted'] = predicted

# estimated probability of admission
validate['predictied_prob_admit'] = predictied_prob[:,1]

# estimated probability of rejection!
validate['predictied_prob_not_admit'] = predictied_prob[:,0]

validate.head(10)

In [ ]:
np.sum(validate['actual'] == validate['predicted']) / len(validate)

In [ ]:
confusion_matrix(validate['actual'], validate['predicted'])


In [ ]:
pd.crosstab(validate['actual'], validate['predicted'], rownames=['Actual'], colnames=['Predicted'], margins=True)

In [ ]:
plt.plot(roc_curve(y_test, predictied_prob[:,1])[0], roc_curve(y_test, predictied_prob[:,1])[1])

In [ ]:
# predicted
predicted = rf.predict(X_test)

# predicted probabilities
predictied_prob = rf.predict_proba(X_test)

# actual
validate = pd.DataFrame(y_test)

validate.columns = ['actual']

validate['predicted'] = predicted

# estimated probability of admission
validate['predictied_prob_admit'] = predictied_prob[:,1]

# estimated probability of rejection!
validate['predictied_prob_not_admit'] = predictied_prob[:,0]

validate.head(10)

In [ ]:
# overal accuracy... be careful!!
np.sum(validate['actual'] == validate['predicted']) / len(validate)

In [ ]:
confusion_matrix(validate['actual'], validate['predicted'])

In [ ]:
pd.crosstab(validate['actual'], validate['predicted'], rownames=['Actual'], colnames=['Predicted'], margins=True)

Note: When predicited_prob_admit > 0.5 we predict that the student will be admitted!!

In [ ]:
plt.plot(roc_curve(y_test, predictied_prob[:,1])[0], roc_curve(y_test, predictied_prob[:,1])[1])

Is this a good model? What about error? We we have just done one shot here... perhaps we can use cross-vailidation to get a better idea of the model performance.

### Variable Importance Measure

In [ ]:
rf.feature_importances_

In [ ]:
important_features = dict(zip(X_train.columns, list(rf.feature_importances_)))

In [ ]:
for feature, importance in important_features.items():
    print(str(feature) + " " + str(round(importance, 2)))